# Deutsch-Jozsa Algorithm #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.thirdparty.render import *;

from src.api.ibm import *;
from src.algorithms import *;

np.random.seed(39102901); # for repeatability

In [ ]:
# NOTE: Avoid repeating this cell.
provider = connect_to_ibm_account(force_reload=False);
display(provider.backends());

In [ ]:
nr_qubits = 4;
oraclenr = np.random.randint(0, 2);
circuit = deutsch_jozsa_algorithm(n=nr_qubits, oraclenr=oraclenr, verbose=True);
display(circuit.draw());

In [ ]:
with CreateBackend(
    provider = provider,
    kind = BACKEND_SIMULATOR.AER,
) as backend:
    shots = 10000;
    circuit = deutsch_jozsa_algorithm(n=nr_qubits, verbose=True);
    job = qk_execute(experiments=circuit, backend=backend, shots=shots);
    result = job.result();
    counts = result.get_counts();
    display(QkVisualisation.plot_histogram(counts));

In [ ]:
with CreateBackend(
    provider = provider,
    nr_qubits = nr_qubits + 1,
    kind = BACKEND.LEAST_BUSY,
) as backend:
    shots = 20000;
    circuit = deutsch_jozsa_algorithm(n=nr_qubits);
    print(f'Least busy backend: \x1b[1m{backend}\x1b[0m (note for later usage).');
    %qiskit_job_watcher
    job = qk_execute(experiments=circuit, backend=backend, shots=shots, optimization_level=3);
    print(f'Note this job id for later usage: \x1b[1m{job.job_id()}\x1b[0m.');

In [ ]:
# User input for previous/current job:
job_index = 'xyz1234';
kind = BACKEND.OSLO;

In [ ]:
job = get_past_job(job_index=job_index, kind=kind, provider=provider);
print(f'Job status: \x1b[92;1m{job.status()}\x1b[0m');
if job.done():
    result = job.result();
    counts = result.get_counts();
    display(QkVisualisation.plot_histogram(counts));